In [1]:
import json
import pymysql

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import re

import pandas as pd

In [2]:
# 크롬 드라이버 설정
global driver
driver = webdriver.Chrome()
global wait
wait = WebDriverWait(driver, 20)

# User-Agent 설정
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
options = webdriver.ChromeOptions()
options.add_argument('user-agent=' + user_agent)

In [3]:
with open('info.json', 'r') as file:
    data = json.load(file)
    db_pw = data['db_info']['pw']

    id = data['user_info']['id']
    pw = data['user_info']['pw']

In [13]:
def __init__(db_pw):

    global conn
    conn = pymysql.connect(host='127.0.0.1', user='root', password=db_pw, db='baekjoon', charset='utf8')

    with conn.cursor() as curs:
        sql="""
        CREATE TABLE IF NOT EXISTS problem_info(
            number INT,
            title VARCHAR(40),
            isSolved BOOLEAN,
            level INT,
            korean BOOLEAN,
            PRIMARY KEY (number)
        );

        """
        curs.execute(sql)

    conn.commit()

In [4]:
def login_solved(id, pw):
    '''
    solved 웹사이트에 접속해서 계정으로 로그인한다.
    '''
    login_url = "https://www.acmicpc.net/login?next=%2Fsso%3Fsso%3Dbm9uY2U9YTM5YWVhYTk4MWNhYjU1YWEwZDRlZDQ4Nzc3NzEzOGM%253D%26sig%3D1ca260b9abbbe4df2b002254dd94a7f31b373ff0270b243408963284f8fcd7a1%26redirect%3Dhttps%253A%252F%252Fsolved.ac%252Fapi%252Fv3%252Fauth%252Fsso%253Fprev%253D%25252F"
    driver.get(login_url)

    # 로그인 페이지에서 계정 정보를 입력하고 클릭한다.
    wait.until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="login_form"]/div[2]/input'))
    )

    user_id = driver.find_element(By.XPATH, '//*[@id="login_form"]/div[2]/input')
    user_pw = driver.find_element(By.XPATH, '//*[@id="login_form"]/div[3]/input')

    user_id.send_keys(id)
    user_pw.send_keys(pw)

    login_bt = driver.find_element(By.XPATH, '//*[@id="submit_button"]')
    login_bt.click()  

    # 백준 계정으로 솔브드에 로그인할지 묻는 화면. 수락을 클릭함
    wait.until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="login_form"]/div[4]/div[2]/a'))
    )

    login_bt_2 = driver.find_element(By.XPATH, '//*[@id="login_form"]/div[4]/div[2]/a')
    login_bt_2.click()

In [5]:
def read_solved():
    '''
    솔브드 레벨을 순회하며 문제 정보를 스크래핑한다.
    '''


    def has_korean(text):
        '''
        텍스트에 한글을 포함하고 있는지 확인하는 코드
        '''
        # 한글 유니코드 범위: AC00-D7AF
        korean_regex = re.compile("[\uac00-\ud7af]+")
        return bool(korean_regex.search(text))


    max_level = 15 # 골드 1

    for level in range(1, max_level+1):
        # 각 레벨의 1페이지에 접속하고 html 텍스트 정보를 파싱함
        url = f'https://solved.ac/problems/level/{level}'
        driver.get(url)
        html = BeautifulSoup(driver.page_source, 'lxml')

        # 마지막 페이지 번호 추출
        last_page = int(html.find_all('a', {'class':'css-1yjorof'})[-1].text)

        # 1페이지의 데이터를 수집함.
        df = pd.DataFrame(columns=['number', 'title','isSolved', 'level', 'korean'])

        rows = html.find_all('tr', {'class':'css-1ojb0xa'})

        for row in range(1, len(rows)):
            info = rows[row].find_all('a', {'class':'css-q9j30p'})
            number = int(info[0].find('span').text)
            title = info[1].find('span', {'class':'__Latex__'}).text
            isSolved = True if rows[row].find_all('span', {'class':'ac'}) else False
            korean = has_korean(title)

            # 데이터 삽입
            df.loc[len(df)] = [number, title, isSolved, level, korean]


        # 2페이지부터 순회하며 데이터 수집
        for page in range(2, last_page + 1):
            pg_url = f'?page={page}'
            driver.get(url+pg_url)
            html = BeautifulSoup(driver.page_source, 'lxml')

            rows = html.find_all('tr', {'class':'css-1ojb0xa'})

            for row in range(1, len(rows)):
                info = rows[row].find_all('a', {'class':'css-q9j30p'})
                number = int(info[0].find('span').text)
                title = info[1].find('span', {'class':'__Latex__'}).text
                isSolved = True if rows[row].find_all('span', {'class':'ac'}) else False
                korean = has_korean(title)

                # 데이터 삽입
                df.loc[len(df)] = [number, title, isSolved, level, korean]
    print("백준 문제 크롤링 완료")            
    return df

In [18]:
def insert_into_db(df):
    '''
    데이터베이스의 데이터를 입력한다.
    '''
    with conn.cursor() as curs:
        for r in df.itertuples(index=False):
            print(r)
            sql=f"INSERT INTO problem_info VALUES ({r.number}, {r.title}, {r.isSolved}, {r.level}, {r.korean})"
            curs.execute(sql)

        conn.commit()
    print(f"[{len(df)}]개의 데이터 DB 저장 완료")

In [14]:
__init__(db_pw)

In [9]:
login_solved(id, pw)

In [15]:
df=read_solved()

백준 문제 크롤링 완료


In [ ]:
conn.close()

In [ ]:

conn = pymysql.connect(host='127.0.0.1', user='root', password=db_pw, db='baekjoon', charset='utf8')
with conn.cursor() as curs:
    for r in df.itertuples(index=False):
        print(r.number, r.title, r.isSolved, r.level, r.korean)
        sql=f"INSERT INTO problem_info VALUES ({r.number}, '{r.title}', {r.isSolved}, {r.level}, {r.korean})"
        # sql=f"INSERT INTO problem_info VALUES (%s, '%s', %s, %s, %s)" %(r.number, r.title, r.isSolved, r.level, r.korean)
        curs.execute(sql)
    conn.commit()

1016 제곱 ㄴㄴ 수 False 15 True
